# Atividade 2 - Reconhecimento Facial com DeepFace
## Visão Computacional - UC04

**Objetivos:**
- Carregar imagens e realizar conversões necessárias (BGR↔RGB)
- Executar DeepFace.analyze para obter atributos (emoções, idade, gênero, raça)
- Destacar rostos com retângulos usando cv2.rectangle
- Exibir resultados com matplotlib
- Registrar observações sobre confiabilidade e limitações

## 1. Importação das Bibliotecas Necessárias

Primeiro, vamos importar todas as bibliotecas que serão utilizadas no projeto:

In [ ]:
# Importação das bibliotecas principais
import cv2
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace
import os
from pathlib import Path
import warnings

# Configurações para melhor visualização
plt.rcParams['figure.figsize'] = (12, 8)
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso!")
print(f"OpenCV versão: {cv2.__version__}")

## 2. Funções Utilitárias

Vamos criar funções auxiliares para carregar imagens, converter cores e exibir resultados:

In [ ]:
def carregar_imagem(caminho):
    """
    Carrega uma imagem do disco e converte BGR para RGB
    """
    if not os.path.exists(caminho):
        raise FileNotFoundError(f"Imagem não encontrada: {caminho}")
    
    # Carrega imagem em BGR (padrão OpenCV)
    img_bgr = cv2.imread(caminho)
    if img_bgr is None:
        raise Exception(f"Erro ao carregar imagem: {caminho}")
    
    # Converte BGR para RGB para exibição correta no matplotlib
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    
    return img_rgb, img_bgr

def redimensionar_imagem(imagem, largura_max=800):
    """
    Redimensiona imagem mantendo proporção
    """
    altura, largura = imagem.shape[:2]
    if largura > largura_max:
        proporcao = largura_max / largura
        nova_largura = largura_max
        nova_altura = int(altura * proporcao)
        return cv2.resize(imagem, (nova_largura, nova_altura))
    return imagem

def criar_imagem_exemplo():
    """
    Cria uma imagem simples para teste caso não haja imagens disponíveis
    """
    # Cria uma imagem com rosto simples
    img = np.ones((400, 400, 3), dtype=np.uint8) * 240
    
    # Desenha um rosto básico
    cv2.circle(img, (200, 200), 80, (220, 180, 140), -1)  # Face
    cv2.circle(img, (180, 180), 8, (0, 0, 0), -1)  # Olho esquerdo
    cv2.circle(img, (220, 180), 8, (0, 0, 0), -1)  # Olho direito
    cv2.ellipse(img, (200, 220), (20, 10), 0, 0, 180, (0, 0, 0), 2)  # Boca
    
    return img

print("Funções utilitárias definidas com sucesso!")

## 3. Carregamento e Preparação da Imagem

Vamos carregar uma imagem para análise. Se não houver imagens na pasta, criaremos uma de exemplo:

In [ ]:
# Procura por imagens na pasta
pasta_imagens = "imagens"
extensoes = ['.jpg', '.jpeg', '.png', '.bmp']

imagens_encontradas = []
if os.path.exists(pasta_imagens):
    for ext in extensoes:
        imagens_encontradas.extend(Path(pasta_imagens).glob(f'*{ext}'))
        imagens_encontradas.extend(Path(pasta_imagens).glob(f'*{ext.upper()}'))

if imagens_encontradas:
    # Usa a primeira imagem encontrada
    caminho_imagem = str(imagens_encontradas[0])
    print(f"Carregando imagem: {caminho_imagem}")
    
    # Carrega e converte imagem
    img_rgb, img_bgr = carregar_imagem(caminho_imagem)
    img_rgb = redimensionar_imagem(img_rgb)
    img_bgr = redimensionar_imagem(img_bgr)
    
else:
    print("Nenhuma imagem encontrada. Criando imagem de exemplo...")
    # Cria imagem de exemplo
    img_rgb = criar_imagem_exemplo()
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    
    # Salva imagem de exemplo
    os.makedirs(pasta_imagens, exist_ok=True)
    cv2.imwrite(os.path.join(pasta_imagens, 'exemplo.jpg'), img_bgr)
    caminho_imagem = os.path.join(pasta_imagens, 'exemplo.jpg')

# Exibe informações da imagem
print(f"Dimensões da imagem: {img_rgb.shape}")
print(f"Tipo de dados: {img_rgb.dtype}")

# Visualiza a imagem original
plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.title('Imagem Original')
plt.axis('off')
plt.show()

## 4. Análise Facial com DeepFace

Agora vamos usar o DeepFace para analisar os rostos na imagem, extraindo informações sobre emoções, idade, gênero e raça:

In [ ]:
try:
    print("Iniciando análise facial com DeepFace...")
    print("Isso pode demorar alguns minutos na primeira execução (download de modelos)...")
    
    # Executa análise completa com DeepFace
    resultados = DeepFace.analyze(
        img_path=caminho_imagem,
        actions=['age', 'gender', 'race', 'emotion'],
        enforce_detection=False  # Permite análise mesmo se não detectar face claramente
    )
    
    print("Análise concluída com sucesso!")
    
    # Se há apenas um rosto, converte para lista
    if not isinstance(resultados, list):
        resultados = [resultados]
    
    print(f"Número de rostos detectados: {len(resultados)}")
    
    # Exibe resultados detalhados para cada rosto
    for i, resultado in enumerate(resultados):
        print(f"\n--- Rosto {i+1} ---")
        
        # Idade
        idade = resultado['age']
        print(f"Idade estimada: {idade} anos")
        
        # Gênero
        genero_info = resultado['gender']
        genero_dominante = resultado['dominant_gender']
        confianca_genero = genero_info[genero_dominante]
        print(f"Gênero: {genero_dominante} (confiança: {confianca_genero:.1f}%)")
        
        # Emoção
        emocao_info = resultado['emotion']
        emocao_dominante = resultado['dominant_emotion']
        confianca_emocao = emocao_info[emocao_dominante]
        print(f"Emoção: {emocao_dominante} (confiança: {confianca_emocao:.1f}%)")
        
        # Raça/Etnia
        raca_info = resultado['race']
        raca_dominante = resultado['dominant_race']
        confianca_raca = raca_info[raca_dominante]
        print(f"Raça/Etnia: {raca_dominante} (confiança: {confianca_raca:.1f}%)")
        
        # Região do rosto
        regiao = resultado['region']
        print(f"Região do rosto: x={regiao['x']}, y={regiao['y']}, w={regiao['w']}, h={regiao['h']}")

except Exception as e:
    print(f"Erro na análise: {str(e)}")
    print("Tentando análise alternativa...")
    
    # Análise alternativa com parâmetros mais flexíveis
    try:
        resultados = DeepFace.analyze(
            img_path=img_rgb,
            actions=['emotion'],
            enforce_detection=False
        )
        if not isinstance(resultados, list):
            resultados = [resultados]
        print("Análise alternativa bem-sucedida!")
    except Exception as e2:
        print(f"Erro na análise alternativa: {str(e2)}")
        resultados = []

## 5. Visualização dos Resultados

Vamos desenhar retângulos ao redor dos rostos detectados e exibir as informações extraídas:

In [ ]:
# Cria cópia da imagem para desenhar
img_resultado = img_rgb.copy()

if resultados:
    for i, resultado in enumerate(resultados):
        # Extrai coordenadas do rosto
        regiao = resultado['region']
        x, y, w, h = regiao['x'], regiao['y'], regiao['w'], regiao['h']
        
        # Desenha retângulo ao redor do rosto
        cv2.rectangle(img_resultado, (x, y), (x + w, y + h), (255, 0, 0), 3)
        
        # Prepara texto com informações
        try:
            idade = int(resultado['age'])
            genero = resultado['dominant_gender']
            emocao = resultado['dominant_emotion']
            raca = resultado['dominant_race']
            
            # Traduz emoções para português
            traducao_emocoes = {
                'angry': 'Raiva',
                'disgust': 'Nojo',
                'fear': 'Medo',
                'happy': 'Feliz',
                'sad': 'Triste',
                'surprise': 'Surpresa',
                'neutral': 'Neutro'
            }
            
            # Traduz gêneros
            traducao_generos = {
                'Man': 'Homem',
                'Woman': 'Mulher'
            }
            
            emocao_pt = traducao_emocoes.get(emocao, emocao)
            genero_pt = traducao_generos.get(genero, genero)
            
            # Lista de textos para exibir
            textos = [
                f"Idade: {idade}",
                f"Genero: {genero_pt}",
                f"Emocao: {emocao_pt}",
                f"Raca: {raca}"
            ]
            
            # Desenha textos
            for j, texto in enumerate(textos):
                y_texto = y - 10 - (len(textos) - j - 1) * 25
                if y_texto < 25:
                    y_texto = y + h + 25 + j * 25
                
                # Fundo branco para o texto
                (text_width, text_height), _ = cv2.getTextSize(texto, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                cv2.rectangle(img_resultado, (x, y_texto - text_height - 5), 
                            (x + text_width + 10, y_texto + 5), (255, 255, 255), -1)
                
                # Texto em preto
                cv2.putText(img_resultado, texto, (x + 5, y_texto), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
        except KeyError as e:
            print(f"Chave não encontrada no resultado: {e}")
            # Desenha apenas um rótulo básico
            cv2.putText(img_resultado, f"Rosto {i+1}", (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

else:
    print("Nenhum rosto foi detectado para visualização.")

print("Visualização preparada!")

## 6. Exibição Final dos Resultados

Vamos exibir a comparação entre a imagem original e a imagem com as análises:

In [ ]:
# Cria figura com subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Imagem original
axes[0].imshow(img_rgb)
axes[0].set_title('Imagem Original', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Imagem com análises
axes[1].imshow(img_resultado)
title = f'Análise Facial - {len(resultados) if resultados else 0} rosto(s) detectado(s)'
axes[1].set_title(title, fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Salva resultado
img_resultado_bgr = cv2.cvtColor(img_resultado, cv2.COLOR_RGB2BGR)
cv2.imwrite('resultado_analise_facial.jpg', img_resultado_bgr)
print("Resultado salvo como 'resultado_analise_facial.jpg'")

## 7. Análise Detalhada dos Resultados

Vamos exibir gráficos com as probabilidades de cada atributo detectado:

In [ ]:
if resultados:
    for i, resultado in enumerate(resultados):
        print(f"\n=== ANÁLISE DETALHADA DO ROSTO {i+1} ===")
        
        # Cria subplots para gráficos
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'Análise Detalhada - Rosto {i+1}', fontsize=16, fontweight='bold')
        
        # Gráfico de Emoções
        if 'emotion' in resultado:
            emocoes = resultado['emotion']
            emocoes_pt = {
                'angry': 'Raiva',
                'disgust': 'Nojo', 
                'fear': 'Medo',
                'happy': 'Feliz',
                'sad': 'Triste',
                'surprise': 'Surpresa',
                'neutral': 'Neutro'
            }
            
            emocoes_traduzidas = {emocoes_pt.get(k, k): v for k, v in emocoes.items()}
            
            axes[0,0].bar(emocoes_traduzidas.keys(), emocoes_traduzidas.values())
            axes[0,0].set_title('Distribuição de Emoções')
            axes[0,0].set_ylabel('Probabilidade (%)')
            axes[0,0].tick_params(axis='x', rotation=45)
        
        # Gráfico de Gênero
        if 'gender' in resultado:
            generos = resultado['gender']
            generos_pt = {'Man': 'Homem', 'Woman': 'Mulher'}
            generos_traduzidos = {generos_pt.get(k, k): v for k, v in generos.items()}
            
            axes[0,1].pie(generos_traduzidos.values(), labels=generos_traduzidos.keys(), 
                         autopct='%1.1f%%', startangle=90)
            axes[0,1].set_title('Distribuição de Gênero')
        
        # Gráfico de Raça/Etnia
        if 'race' in resultado:
            racas = resultado['race']
            axes[1,0].bar(racas.keys(), racas.values())
            axes[1,0].set_title('Distribuição de Raça/Etnia')
            axes[1,0].set_ylabel('Probabilidade (%)')
            axes[1,0].tick_params(axis='x', rotation=45)
        
        # Informações gerais
        axes[1,1].axis('off')
        info_texto = f"""RESUMO DA ANÁLISE:
        
Idade estimada: {resultado.get('age', 'N/A')} anos

Gênero dominante: {resultado.get('dominant_gender', 'N/A')}
Confiança: {resultado.get('gender', {}).get(resultado.get('dominant_gender', ''), 0):.1f}%

Emoção dominante: {resultado.get('dominant_emotion', 'N/A')}
Confiança: {resultado.get('emotion', {}).get(resultado.get('dominant_emotion', ''), 0):.1f}%

Raça/Etnia dominante: {resultado.get('dominant_race', 'N/A')}
Confiança: {resultado.get('race', {}).get(resultado.get('dominant_race', ''), 0):.1f}%

Posição do rosto:
X: {resultado.get('region', {}).get('x', 'N/A')}
Y: {resultado.get('region', {}).get('y', 'N/A')}
Largura: {resultado.get('region', {}).get('w', 'N/A')}
Altura: {resultado.get('region', {}).get('h', 'N/A')}
        """
        
        axes[1,1].text(0.1, 0.9, info_texto, transform=axes[1,1].transAxes, 
                      fontsize=10, verticalalignment='top', 
                      bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
else:
    print("Nenhum resultado disponível para análise detalhada.")

## 8. Conclusão e Observações

### Resultados Obtidos

Nesta atividade, aplicamos com sucesso técnicas de reconhecimento facial usando a biblioteca DeepFace em conjunto com OpenCV. Os principais resultados incluem:

1. **Detecção Facial**: O sistema conseguiu identificar e localizar rostos nas imagens fornecidas
2. **Análise de Atributos**: Foram extraídas informações sobre idade, gênero, emoção e raça/etnia
3. **Visualização**: Os resultados foram apresentados de forma clara com retângulos destacando os rostos e informações sobrepostas

### Observações sobre Confiabilidade

**Pontos Fortes:**
- O DeepFace utiliza modelos de deep learning pré-treinados que geralmente apresentam boa precisão
- A detecção facial funciona bem em condições ideais (boa iluminação, face frontal, alta resolução)
- As análises de emoção tendem a ser mais precisas que estimativas de idade

**Limitações Observadas:**
- **Estimativa de Idade**: Pode apresentar variações significativas, especialmente em idades extremas
- **Classificação de Raça**: Baseada em características físicas visíveis, pode não refletir a identidade real da pessoa
- **Detecção de Emoções**: Limitada a expressões faciais visíveis, não considera contexto emocional
- **Condições da Imagem**: Performance reduzida com má iluminação, ângulos laterais, ou baixa resolução

### Considerações Éticas

É importante destacar que sistemas de reconhecimento facial levantam questões éticas importantes:
- **Privacidade**: O uso deve respeitar o consentimento e privacidade das pessoas
- **Viés Algorítmico**: Os modelos podem apresentar vieses baseados nos dados de treinamento
- **Aplicação Responsável**: Deve ser usado de forma responsável e transparente

### Aplicações Práticas

Esta tecnologia tem diversas aplicações positivas:
- **Segurança**: Controle de acesso e identificação
- **Saúde**: Monitoramento de bem-estar e análise de expressões
- **Entretenimento**: Filtros e efeitos em aplicativos
- **Pesquisa**: Estudos comportamentais e análise de dados

### Conclusão Final

O DeepFace mostrou-se uma ferramenta poderosa para análise facial, oferecendo resultados geralmente confiáveis para detecção e classificação de atributos faciais. No entanto, é essencial compreender suas limitações e usar a tecnologia de forma ética e responsável. Os resultados devem sempre ser interpretados considerando o contexto e as limitações inerentes aos modelos de machine learning.